In [147]:
!pip install s3fs

import boto3
import sagemaker
from sagemaker.session import Session
# from sagemaker.feature_store.feature_group import FeatureGroup

region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)
# featuregroup_name = 'transactions-fg-manual-ingest'

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)
# feature_group = FeatureGroup(name=featuregroup_name, sagemaker_session=feature_store_session)

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [148]:
# record_identifier_feature_name = "ZIP"
# event_time_feature_name = "EventTime"

# change the bucket name to your desired bucket name 
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = ' hc-ds-pre-aca-zip'

print(default_s3_bucket_name)

# fg_test.create(
#     s3_uri=f"s3://{bucket}/{s3_folder}",
#     record_identifier_name=record_identifier_feature_name,
#     event_time_feature_name=event_time_feature_name,
#     role_arn=role,
#     enable_online_store=False)


sagemaker-us-east-1-915124832670


In [149]:
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print (role)

arn:aws:iam::915124832670:role/hc-sagemaker-default-execution-role


In [150]:
import pandas as pd

fip = pd.read_csv("zip_county_fips.csv")
fip['COUNTYNAME'] = fip['COUNTYNAME'].str.replace(' County','')
fip.columns = fip.columns.str.replace(' ','_')
fip.rename(columns={'COUNTYNAME': 'COUNTY', 'STCOUNTYFP':'FIP'}, inplace=True)
fip.columns = fip.columns.str.replace(' ','_')
fip.columns= fip.columns.str.lower()
fip.county = fip.county.str.strip().str.upper()
fip['county'] = fip['county'].str.replace(' PARISH','')
del fip['classfp']
fip

,zip,county,state,fip
0,36003,AUTAUGA,AL,1001
1,36006,AUTAUGA,AL,1001
2,36008,AUTAUGA,AL,1001
3,36022,AUTAUGA,AL,1001
4,36051,AUTAUGA,AL,1001
...,...,...,...,...
54201,820,ST. CROIX ISLAND,VI,78010
54202,840,ST. CROIX ISLAND,VI,78010
54203,850,ST. CROIX ISLAND,VI,78010
54204,830,ST. JOHN ISLAND,VI,78020


In [151]:
import time

current_time_sec = int(round(time.time()))
# append EventTime feature
fip['EventTime'] = pd.Series([current_time_sec]*len(fip), dtype="float64")

In [152]:
def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == 'object':
            data_frame[label] = data_frame[label].astype("str").astype("string")

# cast object dtype to string. The SageMaker Feature Store Python SDK will then map the string dtype to String feature type.
cast_object_to_string(fip)

In [153]:
from time import gmtime, strftime, sleep

tier_feature_group_name = 'hc-ds-pre-aca-zip' + strftime('%d-%H-%M-%S', gmtime())
# # fg_timestamp = strftime("%Y-%m-%d'T'%H:%M:%SZ", gmtime())
# df['EventTime'] = zip_feature_group_name

In [154]:
from sagemaker.feature_store.feature_group import FeatureGroup

tier_feature_group = FeatureGroup(name=tier_feature_group_name, sagemaker_session=feature_store_session)

In [155]:
record_identifier_feature_name = "zip"
event_time_feature_name = "EventTime"

In [156]:
tier_feature_group.load_feature_definitions(data_frame=fip); # output is suppressed

In [157]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")
tier_feature_group.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}", #offline feature store bucket
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True
)
wait_for_feature_group_creation_complete(feature_group=tier_feature_group)

Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup hc-ds-pre-aca-zip29-13-32-09 successfully created.


In [178]:
zip_feature_group.describe()


{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:915124832670:feature-group/zip-feature-group-25-18-27-24',
 'FeatureGroupName': 'zip-feature-group-25-18-27-24',
 'RecordIdentifierFeatureName': 'ZIP',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'ZIP', 'FeatureType': 'Integral'},
  {'FeatureName': 'COUNTYNAME', 'FeatureType': 'String'},
  {'FeatureName': 'STATE', 'FeatureType': 'String'},
  {'FeatureName': 'STCOUNTYFP', 'FeatureType': 'Integral'},
  {'FeatureName': 'CLASSFP', 'FeatureType': 'String'},
  {'FeatureName': 'EventTime', 'FeatureType': 'String'}],
 'CreationTime': datetime.datetime(2022, 3, 25, 18, 27, 39, 750000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://sagemaker-us-east-1-915124832670/ hc-ds-pre-aca-zipdata',
   'ResolvedOutputS3Uri': 's3://sagemaker-us-east-1-915124832670/ hc-ds-pre-aca-zipdata/915124832670/sagemaker/us-east-1/offline-store/zip-feature-

In [165]:
sagemaker.Session().boto_session.client(
    "sagemaker", region_name=region
).list_feature_groups() 

{'FeatureGroupSummaries': [{'FeatureGroupName': 'zipcode-feature-group-09-21-20-34',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:915124832670:feature-group/zipcode-feature-group-09-21-20-34',
   'CreationTime': datetime.datetime(2021, 11, 9, 21, 22, 23, 730000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'zipcode-feature-group-09-00-44-20',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:915124832670:feature-group/zipcode-feature-group-09-00-44-20',
   'CreationTime': datetime.datetime(2021, 5, 9, 0, 44, 54, 606000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'zipcode-feature-group-05-17-45-21',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:915124832670:feature-group/zipcode-feature-group-05-17-45-21',
   'CreationTime': datetime.datetime(2021, 5, 5, 17, 54, 56, 88000, tzinfo=tzlocal()),
   'FeatureGroupStatus

In [166]:
tier_feature_group.ingest(data_frame=fip, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='hc-ds-pre-aca-zip29-13-32-09', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7efec4f13510>, max_workers=3, max_processes=1, _async_result=<multiprocess.pool.MapResult object at 0x7efed6c5edd0>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

In [176]:
customer_id = 14202
customers_feature_group_name = 'hc-ds-pre-aca-mrktshr25-20-10-04'
sample_record = sagemaker.Session().boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).get_record(
    FeatureGroupName=customers_feature_group_name, RecordIdentifierValueAsString=str(customer_id)
)

In [177]:
sample_record


{'ResponseMetadata': {'RequestId': '5c4fed5e-94e2-429e-a233-66ebd05fed63',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5c4fed5e-94e2-429e-a233-66ebd05fed63',
   'content-type': 'application/json',
   'content-length': '15',
   'date': 'Tue, 29 Mar 2022 14:08:26 GMT'},
  'RetryAttempts': 0}}

In [162]:
# record_identifier_value = str(14202)
# featurestore_runtime.get_record(FeatureGroupName='hc-ds-pre-aca-zip29-13-32-09', RecordIdentifierValueAsString=zip)

ParamValidationError: Parameter validation failed:
Invalid type for parameter RecordIdentifierValueAsString, value: <class 'zip'>, type: <class 'type'>, valid types: <class 'str'>